<a href="https://colab.research.google.com/github/markhalka/Paper_Imlimentations/blob/main/Dyna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import math
from collections import defaultdict
import random
from queue import PriorityQueue

"""
A work-in-progress implimentation of "Dyna-Style Planning with Linear Function Approximation and Prioritized Sweeping" by Sutton et. al
"""

# define parameters as in the paper:
EPS = 0.3
GAMMA = 1
LR = 0.01
MODEL_LR = 0.1
MAX_STEPS = 300
FEAT_SIZE = 16
ENV_NAME = "FrozenLake-v0"
class Dyna():
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.act_size = self.env.action_space.n
        self.obs_size = self.env.observation_space.n
        self.state = self.env.reset()
        self.eps = EPS
        self.gamma = GAMMA
        self.lr = LR
        self.model_lr = MODEL_LR
        self.model = defaultdict(tuple)
        self.b = np.zeros((self.act_size, self.obs_size))
        self.F = np.zeros((self.act_size, self.obs_size, self.obs_size))
        self.q_vals = np.zeros((self.obs_size, self.act_size))
        self.MAX_STEPS = MAX_STEPS
        self.queue = PriorityQueue(maxsize=100)
    
    def get_action(self, state):
        if np.random.random() < self.eps:
            return self.env.action_space.sample()
        else:
            return self.get_best_action(state)

    def get_best_action(self, state):
        return np.argmax([self.q_vals[state, i] for i in range(self.act_size)])

    def get_feat(self, state):
        feat = np.zeros(FEAT_SIZE)
        feat[state] = 1
        return feat

    def update_q_vals(self, state, action, next_s, reward):
        td_error = reward + self.gamma * self.q_vals[next_s, self.get_best_action(next_s)] - self.q_vals[state, action]
        self.q_vals[state, action] += self.lr * td_error
        if abs(td_error) > 0.001 and not self.queue.full():
            self.queue.put(state, td_error)

    def update_F(self, next_feat, feat, action):
        self.F[action] = self.F[action] + self.model_lr * np.matmul((next_feat - np.matmul(self.F[action], feat)), feat.T)

    def update_b(self, r, feat, action):
        self.b[action] = self.b[action] + self.model_lr * (r - np.matmul(self.b[action].T, feat)) * feat

    def update_model(self, state, action, next_s, reward):
        feat = self.get_feat(state)
        next_feat = self.get_feat(next_s)
        self.update_F(next_feat, feat, action)
        self.update_b(reward, feat, action)
    
    def step(self):
        done = False
        steps = 0
        feat = self.get_feat(self.state)
        while not done and steps < self.MAX_STEPS:
            steps += 1
            action = self.get_action(self.state)
            next_s, reward, done, _ = self.env.step(action)
            self.update_q_vals(self.state, action, next_s, reward)
            self.update_model(self.state, action, next_s, reward)
            self.state = self.env.reset() if done else next_s
        self.planning(5)

    def run_test(self):
        self.state = self.env.reset()
        done = False
        steps = 0
        total_reward = 0.0
        while not done and steps < self.MAX_STEPS:
            steps += 1
            action = self.get_best_action(self.state)
            next_s, reward, done, _ = self.env.step(action)
            total_reward += reward
            self.state = self.env.reset() if done else next_s
        return total_reward

    def test(self, n_test):
        total_reward = 0.0
        for i in range(n_test):
            total_reward += self.run_test()
        return total_reward / n_test
        
    def planning(self, planning_steps):
        for steps in range(planning_steps):
            if self.queue.empty():
                return
            i = int(self.queue.get())
            for j in range(16):
                if j == i:
                    continue
                a = np.argmax(self.F[:, i, j])
                if self.F[a, i, j] > 0.01:
                    current_a = np.argmax(self.q_vals[i, :])
                    max_q = np.max(self.q_vals[j, :])
                    td_error = np.max(self.b[:, i] + self.gamma * self.F[:, i, j] * max_q) - self.q_vals[i, current_a]
                    self.q_vals[i, current_a] += self.lr * td_error
                    if not self.queue.full() and abs(td_error) > 1e-2:
                        self.queue.put(j, td_error)

agent = Dyna()

for i in range(100000):
    agent.step()
    if i % 1000 == 0:
        print(agent.test(10))